To store the information for a "room" (/point in the story), we need to store
1. Name/description/options (text)
2. Number of outcomes (?) 
3. Location of room ~structs where those outcomes lead

What about randomness?
- There could be two types of room, one w/user input outcomes and one w/dice-based outcomes
- So regular room:
    * Name/description/options (text) (some fixed length, input with .STRINGC) (or: not a fixed length, but have a variable for length so you know how far to jump to get to the other information?) (this could be last so that the long length of the string doesn't cause problems?)
    * Variable indicating non-random
    * Number of outcomes (unless fixed?) (less than or equal to 3 maybe?)
    *  - Address of room where outcome 1 sends you
       - Outcome 2
       - Outcome 3 ..
- And random room:
     * Name/description/options (text)
     * Variable indicating random
     * Number of outcomes/check difficulty? (Do we want mixed successes or just success/failure?)
     * Addresses to which these outcomes send you
- Then we would need another variable for whether a room is dice-based
- And what stat it uses
- What actual dice do we want? What is the range of stats? What are the ranges of the checks? What are the stats?

Room types:
User input
Random + stat
Based on variable

In [ ]:
;; R6 will store "going" addresses while going to push/pop
;; R7 will store "return" addresses while going to push/pop
;; R5 will be used in push/pop to point to top of stack, so won't be saved

;; To call a subroutine: 
;;     LEA R6, [SUBROUTINE LABEL]
;;     JSR PUSH

;; Before returning from a subroutine: 
;;     LEA R0, POP
;;     JMP R0

;; Path: 
;; Store address of subroutine in R6
;; JSR to push, which stores starting point in R7
;; Push does JMP R6 to get to subroutine
;; Subroutine JMPs to pop
;; Pop RETs to starting point 

;; Eventually, we may need to alter some of these instructions as things get too far apart for JSR or w/e.

In [14]:
;; ROOM/STATE STRUCTURE
;; 1 memory location: which_quality (which of HP/stats the room changes)
;; 1 memory location: change_amt    (how much said stat is changed by)
;; 1 memory location: room_type     (0 for user-input directed room, 1 for dice room, 2 for variable room)

;; Then if user-input room: 
;; 3 memory locations: addresses of destinations for 1, 2, and 3 inputs
;; 1 memory location: Placeholder 

;; Or if dice room: 
;; 1 memory location: Stat to check
;; 1 memory location: Difficulty of check
;; 1 memory location: Destination if success
;; 1 memory location: Destination if failure

;; Or if variable room:
;; 1 memory location: Address of variable to check 
;; 1 memory location: Value of that variable that counts as a success
;; 1 memory location: Destination if success
;; 1 memory location: Destination if failure

;; Then finally:
;; Variable number of memory locations: Room description in compressed format

Assembled! Use %dis or %dump to examine; use %exe to run.


In [35]:
.ORIG x3000
BRnzp LOOP_START

;;-----------------------------------------------------------------
;; Stack setup
;; Setup for the stack 
Top:    .FILL Stack
Stack:  .BLKW #100

PUSH:   LD R5, Top
        STR R0, R5, #0      ;; Save R0 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R1, R5, #0      ;; Save R1 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R2, R5, #0      ;; Save R2 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R3, R5, #0      ;; Save R3 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R4, R5, #0      ;; Save R4 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R6, R5, #0      ;; Save R6 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R7, R5, #0      ;; Save R7 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        ST R5, Top
        JMP R6
        
POP:    LD R5, Top
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R7, R5, #0      ;; Load stack into R7
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R6, R5, #0      ;; Load Stack into R6
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R4, R5, #0      ;; Load Stack into R4
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R3, R5, #0      ;; Load Stack into R3
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R2, R5, #0      ;; Load Stack into R2
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R1, R5, #0      ;; Load Stack into R1
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R0, R5, #0      ;; Load Stack into R0
        ST R5, Top
        RET

;; Variable setup
LOC: .FILL ROOM_UINPUT
NAME: .STRINGC "Ruby"
HP: .FILL #60

;;-----------------------------------------------------------------
;; Main game loop
LOOP_START: 

    LD R1, LOC
    
    ADD R0, R1, #7 ;; Load R0 with address of room's description
    PUTSP          ;; Print room description

    LDR R2, R1, #0 ;; Load R1 with which_quality variable
    BRz NO_CHANGE  ;; If which_quality is zero, skip stat change
    ADD R2, R2, #-1 
    BRnp NOT_HP    ;; If which_quality is one, change HP by appropriate amount
    LDR R2, R1, #1 ;; Load change_amt into R1
    LD R3, HP      ;; Load stat to change (HP) into R2
    ADD R3, R3, R2 ;; Add change_amt to HP in R2
    ST R3, HP      ;; Store back in HP
NOT_HP:
NO_CHANGE:
    LDR R2, R1, #2 ;; Load room_type into R1
    ADD R2, R2, #-1 
    BRn UINPUT
    ;;BRz DICE
    ;;BRp VARIABLE   ;; Break to appropriate room type

UINPUT: 
    LEA R6, CHOICE
    JSR PUSH

BRnzp LOOP_START

HALT

;; ------------------------------------------------------------------
;; Subroutine to get a number from 1-3 from the user
;; Copied from Anita's code in printing_text

CHOICE:   
          IN
          LD R2, LOC
          LD R1, ASCII     ;; Load inverted ASCII conversion constant into R1
          ADD R0, R0, R1   ;; Add it to convert ASCII value into a number
          ADD R0, R0, #-2
          BRn CH1 ;; user chose 1
          BRz CH2 ;; user chose 2
          BRp CH3 ;; user chose 3
          
CH1:      ADD R2, R2, #3
          BRnzp DONE
CH2:      ADD R2, R2, #4
          BRnzp DONE
CH3:      ADD R2, R2, #5
          BRnzp DONE
          
DONE:     LD LOC, R2 ;; Load the data at R2, aka the address of the #[choice] room, into LOC
          LEA R0, POP
          JMP R0

;; Data
ASCII:   .FILL x-30

;; ------------------------------------------------------------------
;; Rooms data
ROOM_UINPUT: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #0 ;; 0: Based on user input
    .FILL ROOM_UINPUT
    .FILL ROOM_UINPUT
    .FILL ROOM_UINPUT ;; Three choices all lead back here
    .FILL #0          ;; Placeholder
    .STRINGC "Here we are in an example room. Enter 1 to jump in place, 2 to do a jumping jack, or 3 to spin in a circle\n"
    
ROOM_RANDOM: 
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #1 ;; 1: Based on dice roll
    .FILL #1 ;; Stat for dice roll: ???? 
    .FILL #1 ;; Difficulty of check
    .FILL ROOM_RANDOM ;; Destination if success
    .FILL ROOM_RANDOM ;; Destination if failure
    .STRINGC "This room decides where to send you based on a dice roll. But it still only sends you back here."
    
ROOM_VARIABLE:
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #2 ;; 2: Based on variable
    .FILL #0 ;; VAR_ADDRESS ;; What variable to check
    .FILL #0 ;; VALUE       ;; What value counts as a success
    .FILL ROOM_VARIABLE ;; Destination if success
    .FILL ROOM_VARIABLE ;; Destination if failure
    .STRINGC "This room checks the value of a variable to determine where to send you." 
    
.END
    
    

Assembled! Use %dis or %dump to examine; use %exe to run.


In [36]:
%exe

Here we are in an example room. Enter 1 to jump in place, 2 to do a jumping jack, or 3 to spin in a circle

Input a character> 2
2
Here we are in an example room. Enter 1 to jump in place, 2 to do a jumping jack, or 3 to spin in a circle

Input a character> 1
1
Here we are in an example room. Enter 1 to jump in place, 2 to do a jumping jack, or 3 to spin in a circle

Input a character> 3
3
Here we are in an example room. Enter 1 to jump in place, 2 to do a jumping jack, or 3 to spin in a circle

Input a character> 

Keyboard Interrupt!
